# Detect Vessel Collisions
Now that we have our pipeline for taking a video of a microscope view of the brain during surgery and performing crops, color thresholding to produce masks, and recombining those back into gifs, its time to begin constructing the simulation in which our learner will live!

The first and most critical component of our simulation is to detect when the agent has selected a "lacing" site that collides with blood vasculature. This is my testing ground to figure out how to implement that

In [2]:
# ROS Kinetic Interferes with cv2 import

import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [72]:
import math
import cv2
import argparse
import numpy as np
from numpy import savetxt
from fastai.vision import *
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
%matplotlib inline

import os 
import glob
import imageio

In [4]:
# Useful function that was constructed the preserve the order of images since each image is related by the time
# dimension. 
def order_files_by_date(path_to_folder, file_type):
    files = glob.glob("%s*%s" % (path_to_folder, file_type))
    files.sort(key=os.path.getmtime)
    return files

In [56]:
top_squares = order_files_by_date(
    "/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/square_segmentation_crops/top_crop/", 
    ".jpg")

left_squares = order_files_by_date(
    "/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/square_segmentation_crops/left_crop/", 
    ".jpg")

right_squares =order_files_by_date(
    "/home/andrew/Github/neuralink-bot/Image_Segmentation/segmented_images/square_segmentation_crops/right_crop/", 
    ".jpg")

# Load Test Image

In [69]:
def get_example(image_files, index):
    img = cv2.imread(image_files[index])
    print(f"Shape: {img.shape}, Size: {img.size}")
    return img

In [102]:
vessel_img = get_example(left_squares, 0)

Shape: (150, 150, 3), Size: 67500


In [73]:
def euc_dist(p1, p2):
    return  math.sqrt( ((p1[0]-p2[0])**2)+((p1[1]-p2[1])**2) )

In [93]:
def black_pixel_bool(vessel_img, i, j):
    if img[i][j][0] == 255 and img[i][j][1] == 255 and img[i][j][2] == 255:
        return True
    else:
        return False

In [100]:
thread_x = 95
thread_y = 95

distances = []

def compute_nearest_vessel(thread, vessel_img):
    for i in range(len(vessel_img)):
        for j in range(len(vessel_img)):
            if black_pixel_bool(vessel_img, i , j) == True:
                result = euc_dist([i, j], [thread_x, thread_y]) 
                distances.append(result)

In [103]:
compute_nearest_vessel('thread', vessel_img)

In [105]:
print(np.argmin(distances))

11931
